# Tidy Tuesday Project for September 17th, 2024

Hello there, folks!  I am coming at you with my first #tidytuesday project.  I am currently learning the Julia programming language

## The Shakespeare Dialogue Dataset